In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from pathlib import Path
from time import sleep

DRIVER = Path('./geckodriver').absolute()

In [2]:
# access IUCN webpage with selenium
URL = 'https://www.iucnredlist.org/search/list'
driver = webdriver.Firefox(executable_path=DRIVER)
driver.get(URL)

In [3]:
# get filters
filters = driver.find_element_by_class_name('filter')

In [4]:
# select only animals
filter_taxonomy = filters.find_elements_by_xpath("//*[text()='Taxonomy']")
filter_taxonomy[0].click()
sleep(2)
filter_animals = filters.find_elements_by_xpath("//*[text()='Animalia']")
filter_animals[0].click()

In [5]:
# filter country legends
filter_country_legends = filters.find_elements_by_xpath("//*[text()='Country Legends']")[0]
filter_country_legends.click()
sleep(2)
filter_legends_items = ['Extant & Reintroduced', 'Extinct', 'Extinct & Reintroduced', 'Possibly Extinct', 'Possibly Extinct & Reintroduced']
for item in filter_legends_items:
    selection = filters.find_elements_by_xpath(f"//*[text()='{item}']")[0]
    selection.click()

In [6]:
# filter regions
filter_land_region = filters.find_elements_by_xpath("//*[text()='Land Regions']")[0]
filter_land_region.click()
sleep(2)
# filter for europe
filter_europe_label = filter_land_region = filters.find_elements_by_xpath("//*[text()='Europe']")[0]
filter_europe = filter_europe_label.find_element_by_xpath("..")
filter_europe.click()
# filter for austria
filter_austria = filter_land_region = filters.find_elements_by_xpath("//*[text()='Austria']")[0]
filter_austria.click()
sleep(5)

In [7]:
# get main hatml content
main_content = driver.find_element_by_class_name('layout-page__major')
main_html = main_content.get_attribute('innerHTML')

In [8]:
# extract species info from html li items
def extract_content(html):
    results = []
    # get html with beautifulsoup
    soup = BeautifulSoup(html, 'html.parser')
    species = soup.find_all('li', class_='list-results__item')
    for s in species:
        result = {}
        result['classification'] = s.contents[0].string
        result['name'] = s.contents[2].string
        result['status'] = s.contents[3].string
        result['region'] = s.contents[4].string
        result['level'] = s.contents[5].get('title')
        results.append(result)
    return results

In [9]:
restults_austria = extract_content(main_html)

In [10]:
len(restults_austria)

23

In [11]:
restults_austria

[{'classification': 'animalia — mammalia',
  'name': 'Alces alces',
  'status': 'Increasing',
  'region': 'Global',
  'level': 'Least Concern'},
 {'classification': 'animalia — mammalia',
  'name': 'Capra ibex',
  'status': 'Stable',
  'region': 'Global, Europe',
  'level': 'Least Concern'},
 {'classification': 'animalia — insecta',
  'name': 'Pyrgus cirsii',
  'status': 'Decreasing',
  'region': 'Global, Europe',
  'level': 'Vulnerable'},
 {'classification': 'animalia — aves',
  'name': 'Falco naumanni',
  'status': 'Stable',
  'region': 'Global',
  'level': 'Least Concern'},
 {'classification': 'animalia — mammalia',
  'name': 'Miniopterus schreibersii',
  'status': 'Decreasing',
  'region': 'Global',
  'level': 'Vulnerable'},
 {'classification': 'animalia — aves',
  'name': 'Gypaetus barbatus',
  'status': 'Decreasing',
  'region': 'Global',
  'level': 'Near Threatened'},
 {'classification': 'animalia — actinopterygii',
  'name': 'Huso huso',
  'status': 'Decreasing',
  'region': 'G

In [3]:
# search the "search box" by country and extract return html of main content
# does not work because we have to use filter
def search_country(country):
    search_box.clear()
    search_box.send_keys(country)
    search_button.click()
    # wait for search
    sleep(10)
    main_content = driver.find_element_by_class_name('layout-page__major')
    main_html = main_content.get_attribute('innerHTML')
    return main_html

In [5]:
# test for some countrys
OECD_COUNTRYS = ['Austria', 'Germany', 'Italy']
country_results = {}
for country in OECD_COUNTRYS:
    html = search_country(country)
    country_results[country] = extract_content(html)